# Comparison of similar matflow-damask workflows

This notebook is intended as a guide for comparison of matflow workflows with only one diffrent variable.
For example, slight variation of equiaxed microstructure for two crystal plasticity models with the same texture, or varying texture for two models with identical microstructure.

In [ ]:
# Firstly, load in the required python libraries and functions:
# libraries and functions from the web:
import numpy as np
from matflow import load_workflow
import pickle
from cycler import cycler
import matplotlib.pyplot as plt
# libraries and functions local to this repository:
from utils import import_diff_params
from lattice_strain import lattice_strain
from plotting import plot_latticestrain
from plotting import plot_truestrain_peakint
from plotting import plot_lattice_strain_dist_inc

For this post-processing objective, we need to load in datasets from multiple diffrent matflow workflows.

However, for workflows containing large simulation result data (>20Gbs), **we need to restrict import of data to one use if possible**:

In [ ]:
# define path to completed matflow workflow directories containing workflow.hdf5 files...
workflow_dir_1 = "/mnt/iusers01/jf01/y15576gb/\
postprocessing/\
034_Ti64_TIFUN-T4_TD_910C_DeformY64x_1mms-1_Xprism_2023-03-14-113312/"

workflow_dir_2 = "/mnt/iusers01/jf01/y15576gb/\
postprocessing/\
034_Ti64_TIFUN-T4_TD_910C_DeformY64x_1mms-1_Xprism_2023-03-14-113312/"

# use load_workflow to import the result.hdf5 file into jupyter as a variable...
workflow_1 = load_workflow(workflow_dir_1)
workflow_2 = load_workflow(workflow_dir_2)

# to work on the simulation results specifically, define the volume_element_response as a python dictionary...
ve_response_1 = workflow_1.tasks.simulate_volume_element_loading.elements[0].outputs.volume_element_response
ve_response_2 = workflow_2.tasks.simulate_volume_element_loading.elements[0].outputs.volume_element_response